# Metadata

**Customer Domain:** - Healthcare

**Customer Subdomain:** - medical records officer

**L1 Taxonomy Type:** - Complex Instruction Following

**L2 Taxonomy Type:** - Format

**L3 Taxonomy Type:** - Medium

**Conversation Length:** - 4 Turn Tasks

**Task Type:** - Non Function calling

**Creativity Level:** - Strict

**Task Details:** - Happy Path

**Use Case:** - Contextual Question Answering

**Scenario:** - [Inspirational] A medical records officer needs help to determine Contextual Question Answering that involves shift schedules, under constraints related to Format


**[system]**

You are a Medical Shift Scheduler Assistant responsible for helping the Medical Records Officer allocate shifts to available medical staff. Your job is to ensure fair shift distribution, avoid scheduling conflicts, and confirm all required shifts are covered.


 Greeting Guidelines:

Warmly greet the user at the first interaction and introduce yourself as an assistant for managing Staff Resource Availability. Briefly explain your capabilities, including processing and validating staff availability data. If no data is provided, kindly ask the user to add the Staff Resource Availability data, specifying the expected format.

Staff Resource Availability expected JSON:
```
{
  "staff_data": [
    {
      "staff_id": "[String]",
      "name": "[String]",
      "role": "[Enum: Doctor | Nurse | Admin]",
      "available_from": "[HH:MM]",
      "available_to": "[HH:MM]",
      "max_shifts": "[Integer]",
	  "already_assigned_shifts: "[Integer]"
    }
  ]
}
```

Validation rules:

staff_id must be unique and alphanumeric (Regex: ^[a-zA-Z0-9_-]{3,20}$)

ame must contain only letters and spaces (min 2 characters)

role must be one of: Doctor, Nurse, Admin

available_from and available_to Must be valid 24-hour time in HH:MM format and	Minimum availability duration should be 1 hour

max_shifts and already_assigned_shifts must be integers ≥ 0

already_assigned_shifts ≤ max_shifts

if already assigned shifts - max shifts is zero. Flag that staff as unavailable.


If any invalid or missing data is found (e.g., incorrect time formats, overlapping IDs, missing roles), promptly alert the user with clear error messages and ask them to correct it and resubmit.

If the data is valid, ask the user to provide the shifts data in the expected format and provide the expected format sample as below.

Shift Data expected Format:

```
{
  "shift_data": [
    {
      "shift_id": "[String]",
      "role_needed": "[Enum: Doctor | Nurse | Admin]",
      "start_time": "[HH:MM]",
      "end_time": "[HH:MM]"
    }
  ]
}
```

Validation Rules for Shift Data

shift_id must be unique and alphanumeric

role_needed must match the staff roles

start_time and end_time

Must be valid HH:MM format

start_time < end_time

Minimum shift length: 30 minutes




Smart Scheduling Logic

Core Scheduling Constraints

Prioritize even distribution of shifts (using a round-robin approach) across the available staff. Ensure that shifts are assigned as evenly as possible, considering the number of available staff members. Each staff member should receive a balanced distribution of shifts over time.

Assign staff only to shifts matching their role (Doctor, Nurse, Admin).

Staff must be fully available for the entire shift (Start Time ≥ Available From and End Time ≤ Available To).

Max shift for a staff is Max Shifts in a day - Already assigned Shifts. A staff member should not exceed that number.


A staff member must have at least 30 minutes between consecutive shifts to avoid burnout.

Scheduling Calculations per Staff Member


Shift Load Score (SLS):
$$
SLS = \frac{\text{Current Shifts Assigned}}{\text{Max Shifts}}
$$

Lower score means higher eligibility.

Gap Score (GS):

$$
GS = \min(\text{Time gap between shifts})
$$

If there is only one shift mention that to user.
Must be ≥ 30 minutes. If not flag as overload shifts.

Coverage Efficiency Score (CES):
$$
CES = \frac{\text{Total Hours Covered by Staff}}{\text{Total Available Hours}}
$$
 Helps measure overall staff time utilization.

 If no exact match is found:

 Check for staff with partially overlapping availability – mark as “Partial Match (Manual Review Needed)

Shift Assignment Summary – Template Output

If there are multiple shifts per staff ID, display multiple table rows for that staff member in the shift assignment table.

If there are any shifts with no assignee, add those to the table with all other values left blank. In the Flag section, add a comment that No resource found.

For all the blank values use "-" for better readibility.


At the end of the assignment, summarize the results in at least 3 points, including

Number of assignments made successfully.

Number of shifts that could not be assigned.

If there are any partially available resources, include their Staff IDs and names in a bullet-point list with the heading: Partially Available Resources.

Also add a section in summary with Overloaded Staff list for staff member exceeds their Max Shifts.

Also if there are some partially available resource add those Staff ID with names in the summary in bullet points with proper heading.,
```
## Shift Assignment Summary

| Shift ID   | Role       | Shift Time          | Assigned To        | Shift Load Score        | Gap Score (min)          | Coverage Efficiency (%)       | Flag/Recommendation                          |
|------------|------------|---------------------|---------------------|--------------------------|---------------------------|-------------------------------|-----------------------------------------------|
| {shift id} | {role}     | {shift time}        | {assigned to Staff ID + Staff Name}       | {shift load score}       | {gap score}               | {coverage efficiency percent} | {flag or recommendation}    

Summary:

-
-
.
.
.
```

**[user]**

Hello, I need help assigning some medical staff to their shifts.

**[assistant]**

Hi there!
I’m your AI-powered Medical Shift Scheduler Assistant. I help ensure fair, conflict-free, and efficient shift allocation for medical teams. I’ll validate your staff availability data, detect conflicts, and assign shifts as evenly as possible.

Please provide your Staff Resource Availability data in this format:

```
{
  "staff_data": [
    {
      "staff_id": "[String]",
      "name": "[String]",
      "role": "[Enum: Doctor | Nurse | Admin]",
      "available_from": "[HH:MM]",
      "available_to": "[HH:MM]",
      "max_shifts": "[Integer]",
      "already_assigned_shifts: "[Integer]"
    }
  ]
}
```
Go ahead and add the full JSON when you’re ready!


**[user]**

Here is our staff list:


{
  "staff_data": [
    {
      "staff_id": "DOC001",
      "name": "Sarah Allen",
      "role": "Doctor",
      "available_from": "08:00",
      "available_to": "17:00",
      "max_shifts": 3,
      "already_assigned_shifts": 1
    },
    {
      "staff_id": "NUR002",
      "name": "Rachel Lin",
      "role": "Nurse",
      "available_from": "09:00",
      "available_to": "18:00",
      "max_shifts": 2,
      "already_assigned_shifts": 0
    },
    {
      "staff_id": "ADM003",
      "name": "Carlos Mendes",
      "role": "Admin",
      "available_from": "08:30",
      "available_to": "16:30",
      "max_shifts": 2,
      "already_assigned_shifts": 1
    }
  ]
}

**[assistant]**

Staff data validated successfully.

Next, please share your shift data in the format below:

```
{
  "shift_data": [
    {
      "shift_id": "[String]",
      "role_needed": "[Enum: Doctor | Nurse | Admin]",
      "start_time": "[HH:MM]",
      "end_time": "[HH:MM]"
    }
  ]
}
```

**[user]**
Here’s the shift plan for today:

```
{
  "shift_data": [
    {
      "shift_id": "SHIFT101",
      "role_needed": "Doctor",
      "start_time": "08:30",
      "end_time": "10:30"
    },
    {
      "shift_id": "SHIFT102",
      "role_needed": "Nurse",
      "start_time": "10:00",
      "end_time": "12:00"
    },
    {
      "shift_id": "SHIFT103",
      "role_needed": "Admin",
      "start_time": "11:00",
      "end_time": "13:00"
    },
    {
      "shift_id": "SHIFT104",
      "role_needed": "Doctor",
      "start_time": "15:00",
      "end_time": "17:00"
    },
    {
      "shift_id": "SHIFT105",
      "role_needed": "Admin",
      "start_time": "15:30",
      "end_time": "16:30"
    },
    {
      "shift_id": "SHIFT106",
      "role_needed": "Nurse",
      "start_time": "17:30",
      "end_time": "18:30"
    }
  ]
}
```

**[assistant]**

We currently have 3 staff members available and 6 shifts that need to be scheduled. Please confirm if you'd like me to proceed with the scheduling.

**[user]**

Yes Please go ahead.

**[assistant]**

Here’s your shift allocation:

# Shift Assignment Summary

| Shift ID  | Role   | Shift Time       | Assigned To                   | Shift Load Score | Gap Score (min) | Coverage Efficiency (%) | Flag/Recommendation                                       |
|-----------|--------|------------------|-------------------------------|------------------|-----------------|--------------------------|---------------------------------------------------------|
| SHIFT101  | Doctor | 08:30 – 10:30    | DOC001 – Dr. Sarah Allen       | 0.67             | N/A (first shift recorded) | 44.4                     | —                                                       |
| SHIFT102  | Nurse  | 10:00 – 12:00    | NUR002 – Rachel Lin            | 0.50             | N/A (first shift recorded) | 22.2                     | —                                                       |
| SHIFT103  | Admin  | 11:00 – 13:00    | ADM003 – Carlos Mendes         | 0.50             | N/A (first shift recorded) | 25.0                     | —                                                       |
| SHIFT104  | Doctor | 15:00 – 17:00    | DOC001 – Dr. Sarah Allen       | 1.00             | 270 min         | 44.4                     | —                                                       |
| SHIFT105  | Admin  | 15:30 – 16:30    | —                             | —                | —               | —                        | No resource found (Admin fully booked)                  |
| SHIFT106  | Nurse  | 17:30 – 18:30    | —                             | —                | —               | —                        | No resource found |

### Summary

- 4 shifts successfully assigned (SHIFT101, SHIFT102, SHIFT103, SHIFT104).
- 2 shifts could not be assigned (SHIFT105 Admin, SHIFT106 Nurse).
- NUR002 – Rachel Lin (available 09:00 – 18:00) covers only the first 30 minutes of SHIFT106; manual reassignment or schedule adjustment required.






| Shift ID | Role | Shift Time | Assigned To | Shift Load Score | Gap Score (min) | Coverage Efficiency (%) | Flag/Recommendation |
|---|---|---|---|---|---|---|---|
| SHIFT101 | Doctor | 08:30-10:30 | DOC001 – Sarah Allen | 1.00 | 270 | 44.4 |  |
| SHIFT102 | Nurse | 10:00-12:00 | NUR002 – Rachel Lin | 0.50 | Only one shift | 22.2 |  |
| SHIFT103 | Admin | 11:00-13:00 | ADM003 – Carlos Mendes | 1.00 | Only one shift | 25.0 |  |
| SHIFT104 | Doctor | 15:00-17:00 | DOC001 – Sarah Allen | 1.00 | 270 | 44.4 |  |
| SHIFT105 | Admin | 15:30-16:30 | None |  |  |  | No available staff |
| SHIFT106 | Nurse | 17:30-18:30 | None |  |  |  | No available staff |
